In [ ]:
%load_ext dotenv
%dotenv .env

In [ ]:
from redbox.app import Redbox
from redbox.models.settings import get_settings
from redbox.models.chain import RedboxQuery, RedboxState, AISettings, ChatLLMBackend
from uuid import uuid4
import langchain

In [ ]:
def get_state(user_uuid, prompts, documents, ai_setting):
    q = RedboxQuery(
        question=f"@newroute {prompts[-1]}",
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=prompts[:-1],
        ai_settings=ai_setting,
        permitted_s3_keys=documents,
    )

    return RedboxState(
        request=q,
    )

In [ ]:
env = get_settings()
env = env.model_copy(update={"elastic_root_index": "redbox-data-integration"})
env = env.model_copy(update={"elastic_chunk_alias": "redbox-data-integration-chunk-current"})

ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=False, env=env)
documents = ["natasha.boyse@digital.trade.gov.uk/ai_in_analysis.docx"]

In [ ]:
questions = [
    "Summarise this document with a focus on use of Redbox by analysts. Also find more information on AI and Large language model policy on gov.uk",
    "summarise this document",
    "summarise this document. also, find the definition of large language models based on wikipedia",
    "summarise this document. also, based on the document tell me what other AI assistant tool are used by DBT apart from Redbox",
]
states = []
for question in questions:
    state = get_state(uuid4(), prompts=[question], documents=documents, ai_setting=ai_setting)
    states.append(state)

In [ ]:
langchain.debug = False
planner_outputs = []
repetition = 10
with open("Planner_results.txt", "w") as text_file:
    for state in states:
        text_file.write(
            "##########################################################################################################"
        )
        text_file.write("Question")
        text_file.write("\n\n")
        text_file.write(state.request.question)
        text_file.write("\n\n")
        for rep in range(repetition):
            for chunk in app.graph.stream(state, stream_mode="updates", subgraphs=True):
                if chunk[1].get("planner"):
                    # print(chunk[1]["planner"])
                    planner_output = chunk[1]["planner"]["messages"][-1].content
                    text_file.write("Planner Output " + str(rep))
                    text_file.write("\n\n")
                    text_file.write(planner_output)
                    text_file.write("\n\n")
                    planner_outputs.append(planner_output)